# Decision Tree with ID3 Algorithm

In this project we will implement decision trees using ID3 (Iterative Dichotomizer 3) algorithm. The decision tree is going to be used for classification on the "prison" data set.

## Import Packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Import Dataset

In [2]:
# Load the dataset
file_path = './Data/prison_dataset.csv'
data = pd.read_csv(file_path)
data

,Fiscal Year Released,Recidivism Reporting Year,Race - Ethnicity,Age At Release,Convicting Offense Classification,Convicting Offense Type,Convicting Offense Subtype,Main Supervising District,Release Type,Part of Target Population,Recidivism - Return to Prison numeric
0,2010,2013,White,<45,D Felony,Violent,Other,3JD,Parole,Yes,1
1,2010,2013,White,>45,D Felony,Other,Other,3JD,Parole,Yes,1
2,2010,2013,White,<45,D Felony,Other,Other,5JD,Parole,Yes,1
3,2010,2013,White,>45,Other Felony,Drug,Trafficking,3JD,Parole,Yes,1
4,2010,2013,Black,<45,D Felony,Drug,Trafficking,3JD,Parole,Yes,1
...,...,...,...,...,...,...,...,...,...,...,...
15419,2015,2018,White,<45,Other Felony,Violent,Other,3JD,Discharged End of Sentence,Yes,0
15420,2015,2018,White,<45,D Felony,Other,Other,5JD,Discharged End of Sentence,No,0
15421,2015,2018,Black,<45,Other Felony,Violent,Other,3JD,Discharged End of Sentence,Yes,0
15422,2015,2018,White,<45,D Felony,Drug,Other,5JD,Parole,No,0


## EDA

In [3]:
data.head()

,Fiscal Year Released,Recidivism Reporting Year,Race - Ethnicity,Age At Release,Convicting Offense Classification,Convicting Offense Type,Convicting Offense Subtype,Main Supervising District,Release Type,Part of Target Population,Recidivism - Return to Prison numeric
0,2010,2013,White,<45,D Felony,Violent,Other,3JD,Parole,Yes,1
1,2010,2013,White,>45,D Felony,Other,Other,3JD,Parole,Yes,1
2,2010,2013,White,<45,D Felony,Other,Other,5JD,Parole,Yes,1
3,2010,2013,White,>45,Other Felony,Drug,Trafficking,3JD,Parole,Yes,1
4,2010,2013,Black,<45,D Felony,Drug,Trafficking,3JD,Parole,Yes,1


In [4]:
data.shape

(15424, 11)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15424 entries, 0 to 15423
Data columns (total 11 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   Fiscal Year Released                   15424 non-null  int64 
 1   Recidivism Reporting Year              15424 non-null  int64 
 2   Race - Ethnicity                       15424 non-null  object
 3   Age At Release                         15424 non-null  object
 4   Convicting Offense Classification      15424 non-null  object
 5   Convicting Offense Type                15424 non-null  object
 6   Convicting Offense Subtype             15424 non-null  object
 7   Main Supervising District              15424 non-null  object
 8   Release Type                           15424 non-null  object
 9   Part of Target Population              15424 non-null  object
 10  Recidivism - Return to Prison numeric  15424 non-null  int64 
dtypes: int64(3), ob

In [6]:
data.describe()

,Fiscal Year Released,Recidivism Reporting Year,Recidivism - Return to Prison numeric
count,15424.000000,15424.000000,15424.000000
mean,2013.716610,2016.716610,0.562824
std,1.773688,1.773688,0.496054
min,2010.000000,2013.000000,0.000000
25%,2013.000000,2016.000000,0.000000
50%,2015.000000,2018.000000,1.000000
75%,2015.000000,2018.000000,1.000000
max,2015.000000,2018.000000,1.000000


In [7]:
def calculate_missing_data_percentage(data):
    # Calculate the percentage of missing data for each feature
    missing_data_percentage = data.isnull().mean() * 100
    missing_data_df = pd.DataFrame({
        'Feature': missing_data_percentage.index,
        'Percentage Missing': missing_data_percentage.values
    })
    
    # Sort the DataFrame by percentage missing in descending order
    missing_data_df = missing_data_df.sort_values(by='Percentage Missing', ascending=False)
    
    return missing_data_df

missing_data = calculate_missing_data_percentage(data)
missing_data

,Feature,Percentage Missing
0,Fiscal Year Released,0.0
1,Recidivism Reporting Year,0.0
2,Race - Ethnicity,0.0
3,Age At Release,0.0
4,Convicting Offense Classification,0.0
5,Convicting Offense Type,0.0
6,Convicting Offense Subtype,0.0
7,Main Supervising District,0.0
8,Release Type,0.0
9,Part of Target Population,0.0


In [8]:
unique_values = {column: data[column].unique() for column in data.columns}

unique_values

{'Fiscal Year Released': array([2010, 2013, 2015]),
 'Recidivism Reporting Year': array([2013, 2016, 2018]),
 'Race - Ethnicity': array(['White', 'Black'], dtype=object),
 'Age At Release': array(['<45', '>45'], dtype=object),
 'Convicting Offense Classification': array(['D Felony', 'Other Felony'], dtype=object),
 'Convicting Offense Type': array(['Violent', 'Other', 'Drug'], dtype=object),
 'Convicting Offense Subtype': array(['Other', 'Trafficking'], dtype=object),
 'Main Supervising District': array(['3JD', '5JD'], dtype=object),
 'Release Type': array(['Parole', 'Discharged End of Sentence'], dtype=object),
 'Part of Target Population': array(['Yes', 'No'], dtype=object),
 'Recidivism - Return to Prison numeric': array([1, 0])}

No missing data!

## Preprocessing

We encode the categorical data into numeral ones.

In [9]:
# Encode categorical variables
data_encoded = pd.get_dummies(data, drop_first=True)
data_encoded.head()

,Fiscal Year Released,Recidivism Reporting Year,Recidivism - Return to Prison numeric,Race - Ethnicity_White,Age At Release_>45,Convicting Offense Classification_Other Felony,Convicting Offense Type_Other,Convicting Offense Type_Violent,Convicting Offense Subtype_Trafficking,Main Supervising District_5JD,Release Type_Parole,Part of Target Population_Yes
0,2010,2013,1,1,0,0,0,1,0,0,1,1
1,2010,2013,1,1,1,0,1,0,0,0,1,1
2,2010,2013,1,1,0,0,1,0,0,1,1,1
3,2010,2013,1,1,1,1,0,0,1,0,1,1
4,2010,2013,1,0,0,0,0,0,1,0,1,1


We will also split the dataset into training and test sets with an 80-20 ratio.

In [10]:
def train_test_split(X, y, test_size=0.2, random_state=42):
    np.random.seed(random_state)
    indices = np.random.permutation(len(X))
    test_size = int(len(X) * test_size)
    test_indices = indices[:test_size]
    train_indices = indices[test_size:]
    return X.iloc[train_indices], X.iloc[test_indices], y.iloc[train_indices], y.iloc[test_indices]

# Split the data into features and target variable
X = data_encoded.drop('Recidivism - Return to Prison numeric', axis=1)
y = data_encoded['Recidivism - Return to Prison numeric']

# Split the data into training and testing sets (80-20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((12340, 11), (3084, 11), (12340,), (3084,))

## Decision Tree with ID3

In [11]:
class ID3:
    def __init__(self, max_depth=3):
        self.max_depth = max_depth
        self.tree = None

    def fit(self, X, y):
        self.features = X.columns
        self.tree = self.id3(X, y, depth=0)
    
    def predict(self, X):
        return X.apply(self.predict_row, axis=1)
    
    def id3(self, X, y, depth):
        if depth >= self.max_depth or len(X) == 0 or len(set(y)) == 1:
            return y.mode().iloc[0]

        best_feature = self.best_feature(X, y)
        tree = {best_feature: {}}

        for value in X[best_feature].unique():
            sub_X = X[X[best_feature] == value]
            sub_y = y[X[best_feature] == value]
            subtree = self.id3(sub_X.drop(columns=[best_feature]), sub_y, depth + 1)
            tree[best_feature][value] = subtree

        return tree

    def best_feature(self, X, y):
        info_gain = []
        for feature in X.columns:
            gain = self.information_gain(X[feature], y)
            info_gain.append((gain, feature))
        
        return max(info_gain)[1]

    def information_gain(self, feature, y):
        def entropy(s):
            counts = s.value_counts(normalize=True)
            return -sum(counts * np.log2(counts))

        parent_entropy = entropy(y)
        weighted_entropy = sum((feature == value).mean() * entropy(y[feature == value]) for value in feature.unique())
        
        return parent_entropy - weighted_entropy

    def predict_row(self, row):
        tree = self.tree
        while isinstance(tree, dict):
            feature, branches = list(tree.items())[0]
            tree = branches.get(row[feature], 0)
        return tree

In [12]:
# Train model
id3_tree = ID3(max_depth=3)
id3_tree.fit(X_train, y_train)

# Predict
y_pred = id3_tree.predict(X_test)

In [13]:
# Calculate accuracy
def calculate_accuracy(y_true, y_pred):
    return (y_true == y_pred).mean()

# Calculate confusion matrix
def confusion_matrix(y_true, y_pred):
    tp = fp = tn = fn = 0
    for yt, yp in zip(y_true, y_pred):
        if yt == 1 and yp == 1:
            tp += 1
        elif yt == 0 and yp == 1:
            fp += 1
        elif yt == 0 and yp == 0:
            tn += 1
        elif yt == 1 and yp == 0:
            fn += 1
    return np.array([[tp, fn], [fp, tn]])

In [14]:
accuracy = calculate_accuracy(y_test, y_pred)
confusion_mat = confusion_matrix(y_test, y_pred)

accuracy, confusion_mat
print(f"Accuracy: {accuracy:.3f}")
print(f"Confusion Matrix: \n{confusion_mat}")

Accuracy: 0.721
Confusion Matrix: 
[[1133  594]
 [ 265 1092]]
